In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import sys
sys.path.append('../../')
from analysts_tools.growth import *


#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time

import seaborn as sns

In [2]:
def highlight_max(s):
    '''
    Highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [3]:
#start_of_month = (datetime.datetime.today() - datetime.timedelta(days=180))
start_of_month = '2022-08-01' #start_of_month.replace(day=1).strftime("%Y-%m-%d")
end_last_month = '2022-09-10'#(datetime.datetime.today().replace(day=1) - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

In [4]:
query = """
    SELECT 
        bo.submit_date,
        ffg.close_date,
        TO_CHAR(ffg.close_date, 'YYYY-mm') AS month,
        bo.order_id,
        boi.order_item_id,
        ffg.tracking_code,
        bo.customer_id,
        bo.email_address,
        boi.name,
        CASE 
            WHEN s.site_identifier_value = 'CMX' then ((boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.19)
            WHEN s.site_identifier_value = 'GDL' then ((boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.19)
            WHEN s.site_identifier_value = 'PBC' then ((boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.19)
            WHEN s.site_identifier_value = 'MTY' then ((boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.19)
            WHEN s.site_identifier_value = 'SPO' then ((boi.quantity * foi.step_unit * boi.sale_price)*1.0/3.88)
            WHEN s.site_identifier_value = 'BHZ' then ((boi.quantity * foi.step_unit * boi.sale_price)*1.0/3.88)
            WHEN s.site_identifier_value = 'CWB' then ((boi.quantity * foi.step_unit * boi.sale_price)*1.0/3.88)
            WHEN s.site_identifier_value = 'VCP' then ((boi.quantity * foi.step_unit * boi.sale_price)*1.0/3.88)
        ELSE ((boi.quantity * foi.step_unit * boi.sale_price)*1.0/3000) END AS gmv_usd,
        CASE 
            WHEN s.site_identifier_value = 'CMX' then (COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit),0)*1.0/19.19)
            WHEN s.site_identifier_value = 'GDL' then (COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit),0)*1.0/19.19)
            WHEN s.site_identifier_value = 'PBC' then (COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit),0)*1.0/19.19)
            WHEN s.site_identifier_value = 'MTY' then (COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit),0)*1.0/19.19)
            WHEN s.site_identifier_value = 'SPO' then (COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit),0)*1.0/3.88)
            WHEN s.site_identifier_value = 'BHZ' then (COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit),0)*1.0/3.88)
            WHEN s.site_identifier_value = 'CWB' then (COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit),0)*1.0/3.88)
            WHEN s.site_identifier_value = 'VCP' then (COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit),0)*1.0/3.88)
        ELSE (COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit),0)*1.0/3000) END AS discount_applied,
        boi.quantity* foi.step_unit as cant,
        UPPER(fpro.unit) as unidades,
        bs.sku_id,
        bs.addl_product_id as padre_sku_id,
        s.site_identifier_value as region_code,
        CASE
            WHEN COALESCE(bcat2.name, bcat.name) ILIKE 'Proteínas%' THEN 'Proteínas'
            ELSE COALESCE(bcat2.name, bcat.name) 
        END AS category,
        CASE
            WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
            WHEN bcat.name ILIKE 'Res%' THEN 'Res'
            WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
            ELSE bcat.name
        END AS subcat,
        baid.adjustment_reason
    FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                 fow        ON fow.owner_id = ffg.owner_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_product"               fpro       ON fpro.product_id = bp.product_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_customer"              fc         ON fc.customer_id = bo.customer_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_business_type"         bt         ON fc.business_type_id = bt.business_type_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat2      ON bcx.category_id = bcat2.category_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

    WHERE 
       (ffg.close_date >= '{desde} 00:00:01' AND ffg.close_date <= '{hasta} 23:59:59')
       AND bo.order_status='SUBMITTED'
       AND fo.fb_order_status_id IN (1,6,7,8)
       --AND s.site_identifier_value IN ('ciudad')
       AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
       AND bop.archived = 'N'
       AND (fot.name IS NULL OR fot.name <> 'REFUND')
       AND fc.business_type_id IN (1,4)
       -- FIX SUPER DESCUENTOS
       AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
       AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
    --ORDER BY 1
""".format(desde=start_of_month, hasta=end_last_month)

dataframe = read_connection_data_warehouse.runQuery(query)
dataframe[["gmv_usd", "cant"]] = dataframe[["gmv_usd", "cant"]].astype(float)
dataframe["submit_date"] = pd.to_datetime(dataframe["submit_date"])
dataframe["close_date"] = pd.to_datetime(dataframe["close_date"])

dataframe.to_csv(f'data_ventas_{start_of_month}_{end_last_month}.csv', index=False)

In [5]:
data_ventas = dataframe.copy()

In [6]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",48322,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [7]:
active_skus = frida_products[frida_products.status_frida == 'on']["sku_id"].unique()
data_ventas = data_ventas[data_ventas.sku_id.isin(active_skus)]

data_ventas.shape

(2652895, 19)

In [8]:
frida_products['sku_id'] = frida_products['sku_id'].astype(str)

In [9]:
frida_products_clean = frida_products[(frida_products.status_frida == 'on')].groupby(by= ['id_tarjeta']).apply(lambda x: '#'.join(x["sku_id"].unique())).reset_index()
frida_products_clean = frida_products_clean.rename(columns={0:'skus'})

In [10]:
data_ventas_df = pd.merge(data_ventas,frida_products_clean, left_on='padre_sku_id', right_on='id_tarjeta')#.drop(columns = ['id_tarjeta'])

In [11]:
skus =[434949,434911,553900,430828,430791,430890,430792,430639,430317,430322,455299,363304,424298,369679,382021,399788,391140,370859,370805,351990,252245,252263,352002,428525,258143,428527,263367,258148,324004,304090,112100,112175,163220,260913,363777,351344,98094,168294,154464,141923,247540,397324,284514,363778,370594,290283,250203,250202,269673,250217,526844,289416,441771,250197,289111,254970,195976,304500,195851,195867,221089,196094,220950,221002,212254,25942,519798,519799,585103,527646,218224,-305196,167897,167193,414214,301981,576214,213606,213605,338312,363489,462257,396195,396198,367096,569460,569446,442106,379863,142749,-302220,200608,281192,485884,323995,83616,308809,58813,226074,477741,340889,531513,136082,516839,167142,578272,434919,434987,434895,484125,470439,428403,428402,430377,513497,430406,363253,578143,385435,508090,369863,369431,369301,508091,370741,370892,252265,252255,322489,360089,356790,258305,453798,363896,532065,285539,269745,258142,324003,-314887,168290,235489,153090,363755,97340,513943,513481,73557,363757,518105,526878,260914,249435,494980,535491,250193,526845,526846,334522,334523,570669,524439,289417,249911,254913,254866,289668,390809,195850,211851,221090,220948,277563,276989,290100,519797,395039,-304404,70313,136396,406151,213232,-303630,375344,447190,561460,366840,442105,379870,380645,365404,192891,533293,280862,222614,323998,97203,-300330,-303354,-300531,511896,226189,226209,-302223,477745,124530,124536,169189,324143,581776,516600,367084,434957,554443,554444,470440,476614,430296,553901,430321,430292,363157,363183,385405,369453,385404,369295,369418,371318,370779,370891,369681,443350,306011,438839,526856,453799,419186,165988,141922,247539,165642,513892,351343,581771,484687,487441,566140,289438,249993,250194,503719,249774,250198,249429,495192,218457,534885,197116,220958,220946,515339,515340,212252,393339,117709,438988,167409,162843,-304011,515192,167356,379797,558258,557288,136394,242095,70550,219840,145641,420019,561465,366583,569459,395142,379869,227195,145692,440367,122090,-302529,313145,234359,99017,516601,367085,476613,430350,430405,455298,553913,430352,363155,363203,363171,385389,369429,438645,371185,370778,368410,385443,369680,369864,438644,370807,252253,252257,351996,269746,324005,322423,263751,526855,438841,73947,163207,-314938,165772,197625,108497,397325,89320,112115,284513,518099,160745,240307,165770,504867,363756,494979,494981,361435,388189,334537,441410,269666,250190,495188,495189,452640,452642,289670,534886,197712,211850,531909,524046,195870,195979,212253,368168,66553,162845,585101,213218,80507,527645,534873,290155,557285,568617,220924,167267,-304614,70548,-304617,338311,225825,363454,366859,371196,396203,396205,569453,367462,323996,118004,186939,-300324,59130,73744,80658,394839,542657,477743,528639,212501,254112,327839,178589,102355,401541,516840,-314629,367076,434937,553912,484133,430770,509542,430297,430320,464802,430351,430295,363195,368511,385444,369417,376336,438643,370740,252243,252251,252249,351707,322422,263365,252143,285540,258302,258149,356861,419187,258190,197853,384240,521044,244764,521701,484686,89339,570654,254844,535497,535492,289437,572881,250219,388190,401596,503718,348407,344575,344573,344574,269668,250189,250215,452639,203548,203547,197125,195844,531083,531084,197713,196092,196093,101247,438990,213222,234267,167194,361104,-303930,213215,92952,66518,213205,136392,213604,508903,406149,406150,213230,70542,145422,420017,561466,366581,569445,379876,365403,200610,145410,145690,145702,221589,572883,222598,323999,111047,-303342,-303252,-303249,273998,464554,226083,234361,124651,340895,531512,254113,367072,579081,554034,513496,430290,430501,430404,430291,553914,484127,363191,464004,376337,369865,369294,369303,399789,385498,371319,391142,370893,374161,369289,391141,374393,252261,263750,532066,456106,419185,322490,428518,356539,356860,356788,73648,-314737,144533,284515,164161,526879,112095,518111,414595,504866,521043,246588,244750,513944,518109,581787,249433,306450,306451,249992,269669,250218,250195,388206,249783,441770,250213,250158,390793,197711,252867,197113,195871,117707,-303954,438989,-303522,534874,384802,78599,167898,-304944,70218,447189,561459,367142,491188,367081,371215,379875,380646,258690,258696,63739,132652,533292,63697,491238,123039,491239,120813,422691,239244,533289,419097,400800,124649,222689,322946,490567,147339,130690,136080,574156,369695,367075,435068,430640,430669,553902,484126,513498,430771,430889,455300,476615,363249,369405,369291,370857,370806,368423,385390,385434,369302,370858,428517,356859,258191,322415,306010,438840,258304,258301,453800,74194,-314770,75842,148094,164143,154465,521702,255296,73559,441215,160744,244749,112182,74278,89305,153089,255295,108499,260915,363776,513893,521045,244765,254971,575073,575074,254889,334521,249923,250053,250055,503717,284302,495191,250159,254846,203545,220944,531077,211888,195977,203539,568597,195980,196131,290102,167408,515193,290154,234268,557286,557287,568619,123410,123412,412662,70349,66520,80054,80052,64193,167264,508901,167266,363530,363441,491189,365474,379864,145412,280868,323990,323989,121994,118002,226941,-300327,-303432,59132,98138,88865,-301701,234650,227796,-301707,528638,327840,82996,409396,367073,434967,434983,428400,464801,430502,430376,430316,430827,363165,370742,427542,424299,369406,371184,369293,369407,369419,252259,322414,258110,258111,456105,363897,-314797,575792,412447,240308,484685,75297,97304,133468,148096,235490,526880,570651,570650,513482,168292,249434,535498,487442,250201,269667,249922,250054,570670,570671,503716,249773,250199,250191,269672,250214,249910,289109,289110,428355,452641,289669,531078,211889,568599,304508,197114,282679,196132,162844,-303531,71746,190255,190256,-304389,64195,288939,363451,363439,366551,367101,366584,367082,442107,367461,134238,132650,200035,483143,280865,124716,440369,117735,175699,-303339,24106,313155,533290,366667,505439,234640,227790,283255,322945,490566,57791,503789,227091]

In [12]:
data_ventas_df = data_ventas_df[data_ventas_df.sku_id.isin(skus)]

In [13]:
data_ventas_df['id_tarjeta'] = data_ventas_df['id_tarjeta'].astype(str)

In [14]:
data_ventas_df = data_ventas_df[((data_ventas_df.close_date>='2022-08-01') & (data_ventas_df.close_date<='2022-08-10')) | ((data_ventas_df.close_date>='2022-09-01') & (data_ventas_df.close_date<='2022-09-10'))]

In [16]:
data_ventas_df['mes'] = np.where((data_ventas_df.close_date>='2022-08-01') & (data_ventas_df.close_date<='2022-08-10'),'august', 'september')

In [22]:
def get_info(dff):
    df = dff.groupby(by=['region_code','mes','customer_id','id_tarjeta']).apply(
        lambda d: pd.Series(
            {
                #"name": d['name'].unique(),
                "gmv_usd": d["gmv_usd"].sum(),
                "cant": d["cant"].sum(),
                #"all_clients": d["customer_id"].nunique(),
                #"ARPU": d["gmv_usd"].sum() / d["customer_id"].nunique(),
                #"avg_cant": d["cant"].sum() / d["customer_id"].nunique(),
                #"discount_usd": d["discount_applied"].sum(),
                #"avg_discount": float(d["discount_applied"].sum() / d["customer_id"].nunique()),
                #"% dct": float(d["discount_applied"].sum()) *100.0 / d["gmv_usd"].sum()

            }
        )
    ).reset_index()
    return df

In [23]:
dff = get_info(data_ventas_df)

In [39]:
clients = dff.groupby(by=['region_code','mes','customer_id']).agg({'gmv_usd':np.sum}).reset_index()

In [41]:
def q1(x):
    return x.quantile(0.25)

def q2(x):
    return x.quantile(0.50)

def q3(x):
    return x.quantile(0.75)

def q4(x):
    return x.quantile(1)

f = {'gmv_usd': [q1, q2, q3, q4]}

In [46]:
quantiles = clients.groupby(by=['region_code','mes']).agg(f).droplevel(0, axis=1).reset_index()

In [57]:
clientes_q = pd.merge(clients,quantiles, left_on=['region_code','mes'], right_on=['region_code','mes'])#.drop(columns = ['id_tarjeta'])

In [62]:
dff_q = pd.merge(dff,clientes_q, left_on=['region_code','mes','customer_id'], right_on=['region_code','mes','customer_id']).drop(columns = ['gmv_usd_y','percen','ARPU_perc'])

In [63]:
dff_q

,region_code,mes,customer_id,id_tarjeta,gmv_usd_x,cant,q1,q2,q3,q4
0,BAQ,august,-254651,180812,14.594000,2.0,20.639583,48.599333,101.321166,4147.6000
1,BAQ,august,-254651,183321,3.750000,3.0,20.639583,48.599333,101.321166,4147.6000
2,BAQ,august,-254651,239467,2.800000,3.0,20.639583,48.599333,101.321166,4147.6000
3,BAQ,august,-254651,359445,25.326667,2.0,20.639583,48.599333,101.321166,4147.6000
4,BAQ,august,-254651,60271,5.218667,4.0,20.639583,48.599333,101.321166,4147.6000
...,...,...,...,...,...,...,...,...,...,...
345656,VCP,september,219442052,325936,9.432980,10.0,18.201020,37.731940,76.335030,1759.7938
345657,VCP,september,219442052,328547,4.059270,5.0,18.201020,37.731940,76.335030,1759.7938
345658,VCP,september,219442052,328623,3.340200,3.0,18.201020,37.731940,76.335030,1759.7938
345659,VCP,september,219489459,328538,2.319580,2.0,18.201020,37.731940,76.335030,1759.7938


In [70]:
dff_q['range'] = np.where(
     dff_q['gmv_usd_x'].between(0, dff_q['q1']), 
    'ARPU_25%', 
     np.where(
        dff_q['gmv_usd_x'].between(dff_q['q1'], dff_q['q2']), 
        'ARPU_50%', 
        np.where(
            dff_q['gmv_usd_x'].between(dff_q['q2'], dff_q['q3']), 
        'ARPU_75%',
        'ARPU_100%'
        )
     )
)

In [75]:
final = dff_q.groupby(by=['region_code','mes','range','id_tarjeta']).apply(
    lambda d: pd.Series(
        {
            #"name": d['name'].unique(),
            "gmv_usd": d["gmv_usd_x"].sum(),
            "cant": d["cant"].sum(),
            "all_clients": d["customer_id"].nunique(),
            "ARPU": d["gmv_usd_x"].sum() / d["customer_id"].nunique(),
            "avg_cant": d["cant"].sum() / d["customer_id"].nunique(),
            #"discount_usd": d["discount_applied"].sum(),
            #"avg_discount": float(d["discount_applied"].sum() / d["customer_id"].nunique()),
            #"% dct": float(d["discount_applied"].sum()) *100.0 / d["gmv_usd"].sum()

        }
    )
).reset_index()

In [73]:
final.to_excel("hola.xlsx")